In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os
import getpass

load_dotenv()
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

os.environ["LANGSMITH_TRACING_V2"] = "true"
if not os.getenv("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
    
loader = PyPDFLoader("../knowledge_pdf/masterigrandview.pdf")
pages = loader.load()
len(pages)

# 2. Splitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

docs = text_splitter.split_documents(pages)
embeddings = OpenAIEmbeddings(
model="text-embedding-3-large",
)

persist_directory = '../docs/chroma/'

vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory=persist_directory
)
len(pages)
len(docs)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/admin/.pyenv/versions/3.11.10/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/admin/.pyenv/versions/3.11.10/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/admin/.pyenv/versions/3.11.10/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
  

AttributeError: _ARRAY_API not found

SystemError: <built-in function __import__> returned a result with an exception set

62

In [ ]:
import faiss
import numpy as np
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

# Tạo embeddings cho từng đoạn văn bản
embeddings_list = [embeddings.embed_query(doc.page_content) for doc in docs]

# Chuyển các embedding thành numpy array
embeddings_array = np.array(embeddings_list, dtype=np.float32)

# Kiểm tra số chiều của vector embedding
embedding_dim = embeddings_array.shape[1]

# Tạo chỉ mục FAISS với khoảng cách L2
index = faiss.IndexFlatL2(embedding_dim)

# # Thêm các embedding vào chỉ mục FAISS
index.add(embeddings_array)

# # Lưu chỉ mục FAISS vào file
faiss.write_index(index, "faiss_index.index")

# # Giả sử bạn có một câu truy vấn
query = "Khi nào dự án bàn giao"

# # Chuyển câu truy vấn thành vector embedding
query_vector = embeddings.embed_query(query)

# # Tìm kiếm trong chỉ mục FAISS
D, I = index.search(np.array([query_vector], dtype=np.float32), k=5)

# # D: Khoảng cách giữa câu truy vấn và các đoạn trong tài liệu
# # I: Chỉ mục của các đoạn gần nhất
# print("Khoảng cách (distance):", D)
# print("Chỉ mục (index):", I)
for idx in I[0]:
    print(f"Document: {docs[idx]}")


In [2]:
query = "Giới thiệu tổng quan Masteri Grand View" # Thời gian bàn giao dự kiến thì tìm đúng thông tin, Hỏi thời gian bàn giao thì không chính xác
retrieved_docs = vectordb.similarity_search(query, k=2)
serialized = "\n\n".join(
    (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
    for doc in retrieved_docs
)

In [3]:
print(retrieved_docs)

[Document(metadata={'page': 61, 'source': './mh_docs/masterigrandview.pdf'}, page_content='NHỮNG LÝ DO KHIẾN MASTERI GRAND VIEW \nTRỞ THÀNH LỰA CHỌN HÀNG ĐẦU\n1.\n2.\n3.\n4.\n5.\n6.\nTỌA LẠC TRONG KHU ĐÔ THỊ 117.4HA –\nTHE GLOBAL CITY –\nQUỸ ĐẤT VÀNG TẠI P. AN PHÚ\nLÀ SIÊU PHẨM CAO TẦNG ĐẦU \nTIÊN \nTẠI TRUNG TÂM MỚI\nVỊ TRÍ KIM CƯƠNG TẠI GIAO \nĐIỂM ĐA SẮC MÀU\nĐƯỢC PHÁT TRIỂN BỞI CÁC \nTÊN TUỔI HÀNG ĐẦU THẾ GIỚI\nTHỜI ĐIỂM VÀNG ĐỂ SỞ HỮU \nNHỮNG CĂN HỘ ĐẦU TIÊN TẠI THE \nGLOBAL CITY\nTHỪA HƯỞNG TOÀN BỘ HỆ TIỆN \nÍCH ĐỈNH CAO CỦA \nTHE GLOBAL CITY\n7.\n8.\nSỞ HỮU LOẠI HÌNH 4PN CỰC KHAN \nHIẾM TRÊN THỊ TRƯỜNG\nMỖI SÀN CHỈ 8CĂN'), Document(metadata={'page': 61, 'source': './mh_docs/masterigrandview.pdf'}, page_content='NHỮNG LÝ DO KHIẾN MASTERI GRAND VIEW \nTRỞ THÀNH LỰA CHỌN HÀNG ĐẦU\n1.\n2.\n3.\n4.\n5.\n6.\nTỌA LẠC TRONG KHU ĐÔ THỊ 117.4HA –\nTHE GLOBAL CITY –\nQUỸ ĐẤT VÀNG TẠI P. AN PHÚ\nLÀ SIÊU PHẨM CAO TẦNG ĐẦU \nTIÊN \nTẠI TRUNG TÂM MỚI\nVỊ TRÍ KIM CƯƠNG TẠI GIAO \nĐIỂM ĐA SẮC MÀU

In [4]:
print(serialized)

Source: {'page': 61, 'source': './mh_docs/masterigrandview.pdf'}
Content: NHỮNG LÝ DO KHIẾN MASTERI GRAND VIEW 
TRỞ THÀNH LỰA CHỌN HÀNG ĐẦU
1.
2.
3.
4.
5.
6.
TỌA LẠC TRONG KHU ĐÔ THỊ 117.4HA –
THE GLOBAL CITY –
QUỸ ĐẤT VÀNG TẠI P. AN PHÚ
LÀ SIÊU PHẨM CAO TẦNG ĐẦU 
TIÊN 
TẠI TRUNG TÂM MỚI
VỊ TRÍ KIM CƯƠNG TẠI GIAO 
ĐIỂM ĐA SẮC MÀU
ĐƯỢC PHÁT TRIỂN BỞI CÁC 
TÊN TUỔI HÀNG ĐẦU THẾ GIỚI
THỜI ĐIỂM VÀNG ĐỂ SỞ HỮU 
NHỮNG CĂN HỘ ĐẦU TIÊN TẠI THE 
GLOBAL CITY
THỪA HƯỞNG TOÀN BỘ HỆ TIỆN 
ÍCH ĐỈNH CAO CỦA 
THE GLOBAL CITY
7.
8.
SỞ HỮU LOẠI HÌNH 4PN CỰC KHAN 
HIẾM TRÊN THỊ TRƯỜNG
MỖI SÀN CHỈ 8CĂN

Source: {'page': 61, 'source': './mh_docs/masterigrandview.pdf'}
Content: NHỮNG LÝ DO KHIẾN MASTERI GRAND VIEW 
TRỞ THÀNH LỰA CHỌN HÀNG ĐẦU
1.
2.
3.
4.
5.
6.
TỌA LẠC TRONG KHU ĐÔ THỊ 117.4HA –
THE GLOBAL CITY –
QUỸ ĐẤT VÀNG TẠI P. AN PHÚ
LÀ SIÊU PHẨM CAO TẦNG ĐẦU 
TIÊN 
TẠI TRUNG TÂM MỚI
VỊ TRÍ KIM CƯƠNG TẠI GIAO 
ĐIỂM ĐA SẮC MÀU
ĐƯỢC PHÁT TRIỂN BỞI CÁC 
TÊN TUỔI HÀNG ĐẦU THẾ GIỚI
THỜI ĐIỂM VÀNG ĐỂ SỞ HỮU 